## Imports

In [2]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula
from datetime import datetime, timedelta

## Read-in

In [3]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all')

## Current Week's Values

In [37]:
week_starting = 'Dec. 24th'
week_ending = 'Dec. 30th'
current_week_condo_sales = 60
average_sale_price = 580_000
average_price_psf = 519
average_days_on_market = '73'
current_week_condo_sales_total = '$34.8 million'

## Previous Week's Values

In [104]:
previous_week_condo_sales = 130
previous_week_average_sales_price = 577_000
previous_week_average_psf = 504
previous_week_condo_sales_total = '$75 million'

## Clean Data

In [5]:
df = dsf[0]
df.columns = df.iloc[0]
df = df[1:]
df = df.drop(columns=df.columns[0])
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['building_address_city'] = df['building_address_city'].str.replace('^0', '', regex=True)
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
pattern = r'^(?P<building_name>[\d\s]*[A-Za-z\s]+)\s(?P<address>.+)\s(?P<city>[A-Za-z\s]+)$'
df2 = df['geo_address']
df2 = df2.to_frame()
df2[['building_name', 'address', 'city']] = df2['geo_address'].str.extract(pattern, expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [6]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result
df = df.drop(columns='city')

In [7]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [8]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address_x'].apply(geocode)

In [9]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
# df.at[index#,'col_name']

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=False)

In [12]:
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=False)

In [13]:
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

In [14]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=False)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=False)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [15]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address_x', 'geo_address_y', 'building_name',
       'address', 'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [18]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 26th - December 2nd')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [19]:
m.save('index.html')

## Data snagger

In [20]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [21]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Il Villaggio, 1455 Ocean Dr 1609 Miami | Price $4,650,000 | $2,067 psf | Listing agent: Eloy Carmenate with The Corcoran Group | Buyer's agent: Lisa Van Wagenen with Brown Harris Stevens | Days on market: 78
Least Expensive
Mirador, 1000 1000 West Ave PH25 Miami | Price $935,000 | $781 psf | Listing agent: Jamie Manburg with RightStreet Inc | Buyer's agent: Madison Clivilles with Compass Florida, LLC. | Days on market: 29
Highest Price Per Square Foot
Il Villaggio, 1455 Ocean Dr 1609 Miami | Price $4,650,000 | $2,067 psf | Listing agent: Eloy Carmenate with The Corcoran Group | Buyer's agent: Lisa Van Wagenen with Brown Harris Stevens | Days on market: 78
Lowest Price Per Square Foot
Artech, 2950 NE 188th St 510 | Price $960,000 | $426 psf | Listing agent: Valeriya Hafner with Trust Invest Real Estate Corp | Buyer's agent: Valeriya Hafner with Trust Invest Real Estate Corp | Days on market: 119
Most Days on Market
Brickell Key One, 520 Brickell Key Dr A904 | Price $1,000

In [22]:
df['days_on_market'].mean()

68.3

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [62]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_01052023__TEST_GROUND


# CREATE TEMPLATE 

In [28]:
df['muni_name'] = df['building_address_city'].str.split('\r').str[-1]

In [32]:
muni_set = set(df['muni_name'])

In [82]:
top_sale = df.at[1, 'building_address_city_clean']
second_top_sale = df.at[2,'building_address_city_clean']

In [112]:
# Subtract the smaller value from the larger one
if average_sale_price > previous_week_average_sales_price:
    seo_head = 'rise'
    change = 'higher'
    difference = average_sale_price - previous_week_average_sales_price
else:
    seo_head = 'drop'
    change = 'lower'
    difference = previous_week_average_sales_price - average_sale_price

In [113]:
difference

3000

In [117]:
story_string = f'''
\033[1mHED:\033[0m {df.loc[df['int_Sale_Price'].idxmax()]['building_name']} tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top ten sales ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Condo Sales {seo_head} 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
ART: 

Credit condo.com with the image below and the PDF chart in the story
CONDO_URL_GOES_HERE

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Miami-Dade County’s NEWS PEG HERE.

Brokers closed {current_week_condo_sales} condo sales totaling {current_week_condo_sales_total} from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${previous_week_condo_sales_total}.

Last week’s units sold for an average of {average_sale_price}, {change} than the {previous_week_average_sales_price} from the previous week. The average price per square foot rose/fell to ${average_price_psf} from ${previous_week_average_psf}, according to data from condo.com. Condos closed after an average of {average_days_on_market} days on the market.

For the top 10 sales, prices ranged from {df.at[10,'Sale Price']} to {df.at[1,'Sale Price']}. 

{top_sale[0]}, at {top_sale[1]} in {top_sale[2]} took the top spot with a ${df.at[1,'Sale Price']} million closing. {df.at[1,'Agent']} with {df.at[1,'Listing Broker']} had the listing. {df.at[1,'Listing Broker']} with {df.at[1,'Buyer Broker']} represented the buyer. The sale closed after {df.at[1,'days_on_market']} days on the market at ${df.at[1,'price_per_sqft']} per square foot.

{second_top_sale[0]} in {top_sale[2]}, at {top_sale[1]}, closed for the second highest amount, ${df.at[2,'Sale Price']}, or {df.at[2,'price_per_sqft']}, after {df.at[2,'days_on_market']}. {df.at[2,'Agent']} with {df.at[2,'Listing Broker']} had the listing. {df.at[2,'Buyer Agent']} with {df.at[2,'Buyer Broker']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

In [118]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address_x', 'geo_address_y', 'building_name',
       'address', 'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR',
       'muni_name', 'building_address_city_clean'],
      dtype='object')

In [121]:
print(story_string)

### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")


HED: Il Villaggio tops Miami-Dade weekly condo sales 
DEK: Top ten sales ranged from $935,000.00 to $4,650,000.00
FEATURED HED:
SEO HED: Miami-Dade County Condo Sales rise 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXXXX
ART: 

Credit condo.com with the image below and the PDF chart in the story
CONDO_URL_GOES_HERE

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Key Biscayne, Aventura, Miami, Bay Harbor Islands, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Miami-Dade County’s NEWS PEG HERE.

Brokers closed 60 condo sales totaling $34.8 million from Dec. 24th to Dec. 30th. The previous week, brokers closed 130 condo sales totaling $$75 million.

Last week’s units sold for an average of 580000, higher than the 57

In [58]:
df['building_address_city_clean'] = df['building_address_city'].str.split('\r')

In [60]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,...,building_name,address,geocoded,lat,lon,int_Sale_Price,RANK,COLOR,muni_name,building_address_city_clean
1,Il Villaggio\r1455 Ocean Dr 1609\rMiami Beach,12/29/2023,78,"$4,650,000.00",2250,2066.67,Eloy Carmenate,The Corcoran Group,Lisa Van Wagenen,Brown Harris Stevens,...,Il Villaggio,1455 Ocean Dr 1609 Miami,"(25.7873169, -80.1293566)",25.787317,-80.129357,4650000.0,1,orange,Miami Beach,"[Il Villaggio, 1455 Ocean Dr 1609, Miami Beach]"
2,Icon Brickell\r465 Brickell Ave 2603\rMiami,12/28/2023,84,"$1,200,000.00",1313,913.94,Talita Pinheiro,"Compass Florida, LLC.",Stephanie Bienstock,Blackbook Properties,...,Icon Brickell,465 Brickell Ave 2603,"(25.7689974, -80.18854999999999)",25.768997,-80.188550,1200000.0,2,blue,Miami,"[Icon Brickell, 465 Brickell Ave 2603, Miami]"
3,Commodore Club South Condo\r199 Ocean Lane Dr ...,12/27/2023,9,"$1,175,000.00",1260,932.54,Silvia Caicedo,BHHS EWM Realty,Angela Langlois,BHHS EWM Realty,...,Commodore Club South Condo,199 Ocean Lane Dr 812 Key,"(25.699304, -80.157096)",25.699304,-80.157096,1175000.0,3,blue,Key Biscayne,"[Commodore Club South Condo, 199 Ocean Lane Dr..."
4,One Paraiso\r3131 NE 7th Ave 4205\rMiami,12/29/2023,108,"$1,025,000.00",1088,942.10,Fabrizio Passatore,The Keyes Company,Iman Ajlani,Brown Harris Stevens,...,One Paraiso,3131 NE 7th Ave 4205,"(25.8071448, -80.1856208)",25.807145,-80.185621,1025000.0,4,blue,Miami,"[One Paraiso, 3131 NE 7th Ave 4205, Miami]"
5,Brickell Key One\r520 Brickell Key Dr A904\rMiami,12/28/2023,191,"$1,000,000.00",2001,554.63,Jacqueline Bonnet,"P.U.R.E. Investments, Inc.",Mustafa Altinok,Capital Investments R.E. Corp.,...,Brickell Key One,520 Brickell Key Dr A904,"(25.767054, -80.18486209999999)",25.767054,-80.184862,1000000.0,5,blue,Miami,"[Brickell Key One, 520 Brickell Key Dr A904, M..."
6,Artech\r2950 NE 188th St 510\rAventura,12/29/2023,119,"$960,000.00",2253,426.10,Valeriya Hafner,Trust Invest Real Estate Corp,Valeriya Hafner,Trust Invest Real Estate Corp,...,Artech,2950 NE 188th St 510,"(25.948838, -80.13991399999999)",25.948838,-80.139914,960000.0,6,blue,Aventura,"[Artech, 2950 NE 188th St 510, Aventura]"
7,Paraiso Bay\r650 NE 32 St 2404\rMiami,12/29/2023,24,"$960,000.00",1193,804.69,Maria Davila PA,One Sotheby's Int'l Realty,Carolina Lara-Arashiro,One Sotheby's Int'l Realty,...,Paraiso Bay,650 NE 32 St 2404,"(25.8068939, -80.1862582)",25.806894,-80.186258,960000.0,7,blue,Miami,"[Paraiso Bay, 650 NE 32 St 2404, Miami]"
8,The Atrium at Aventura\r3131 NE 188th St 1-121...,12/27/2023,26,"$950,000.00",1800,527.78,Nadejda Damian,ND Homes Realty LLC,Guillermo Rosman,Miami Zyr Realty Inc,...,The Atrium at Aventura,3131 NE 188th St 1-1212,"(25.9498957, -80.1384516)",25.949896,-80.138452,950000.0,8,blue,Aventura,"[The Atrium at Aventura, 3131 NE 188th St 1-12..."
9,Carroll Walk Condo\r9751 E Bay Harbor Dr 7C\rB...,12/27/2023,15,"$945,000.00",1508,626.66,Oscar Teran,"EXP Realty, LLC",Adam Ziefer,The Corcoran Group,...,Carroll Walk Condo,9751 E Bay Harbor Dr 7C Bay Harbor,"(25.8885049, -80.1295143)",25.888505,-80.129514,945000.0,9,blue,Bay Harbor Islands,"[Carroll Walk Condo, 9751 E Bay Harbor Dr 7C, ..."
10,Mirador 1000\r1000 West Ave PH25\rMiami Beach,12/28/2023,29,"$935,000.00",1197,781.12,Jamie Manburg,RightStreet Inc,Madison Clivilles,"Compass Florida, LLC.",...,Mirador,1000 1000 West Ave PH25 Miami,"(25.7810173, -80.14250179999999)",25.781017,-80.142502,935000.0,10,blue,Miami Beach,"[Mirador 1000, 1000 West Ave PH25, Miami Beach]"
